## Step 1: Model Size Reference

Verifies dual T4 GPU environment (30GB total VRAM) required for deploying large language models (13B-70B parameters) on Kaggle.

In [1]:
# Step 1: Model Size Reference (Updated for Kaggle Limits)
print("="*70)
print("📊 MODEL SIZE REFERENCE FOR DUAL T4 (30GB VRAM)")
print("="*70)

models_table = """
╔═══════════════╦═══════════════╦══════════════╦══════════════╗
║ Model Size    ║ Quantization  ║ VRAM Needed  ║ Kaggle Fit?  ║
╠═══════════════╬═══════════════╬══════════════╬══════════════╣
║ 1-3B          ║ Q4_K_M        ║ ~2-3 GB      ║ ✅ Single GPU ║
║ 7-8B          ║ Q4_K_M        ║ ~5-6 GB      ║ ✅ Single GPU ║
║ 13B           ║ Q4_K_M        ║ ~8-9 GB      ║ ✅ Single GPU ║
║ 13B           ║ Q8_0          ║ ~14-15 GB    ║ ✅ Single GPU ║
║ 32-34B        ║ Q4_K_M        ║ ~20-22 GB    ║ ⚠️ Dual GPU   ║
╚═══════════════╩═══════════════╩══════════════╩══════════════╝

📌 Recommended Models for Kaggle:
• Best Speed: 7-8B Q4_K_M (5-6GB VRAM)
• Best Balance: 13B Q4_K_M (8-9GB VRAM)
• Maximum Quality: 13B Q8_0 (14-15GB VRAM)
"""
print(models_table)

📊 MODEL SIZE REFERENCE FOR DUAL T4 (30GB VRAM)

╔═══════════════╦═══════════════╦══════════════╦══════════════╗
║ Model Size    ║ Quantization  ║ VRAM Needed  ║ Kaggle Fit?  ║
╠═══════════════╬═══════════════╬══════════════╬══════════════╣
║ 1-3B          ║ Q4_K_M        ║ ~2-3 GB      ║ ✅ Single GPU ║
║ 7-8B          ║ Q4_K_M        ║ ~5-6 GB      ║ ✅ Single GPU ║
║ 13B           ║ Q4_K_M        ║ ~8-9 GB      ║ ✅ Single GPU ║
║ 13B           ║ Q8_0          ║ ~14-15 GB    ║ ✅ Single GPU ║
║ 32-34B        ║ Q4_K_M        ║ ~20-22 GB    ║ ⚠️ Dual GPU   ║
╚═══════════════╩═══════════════╩══════════════╩══════════════╝

📌 Recommended Models for Kaggle:
• Best Speed: 7-8B Q4_K_M (5-6GB VRAM)
• Best Balance: 13B Q4_K_M (8-9GB VRAM)
• Maximum Quality: 13B Q8_0 (14-15GB VRAM)



Installs llamatelemetry v0.1.0 with multi-GPU tensor-split support for distributing large models across dual T4 GPUs.

In [2]:
!nvidia-smi --query-gpu=index,name,memory.total,memory.free --format=csv

index, name, memory.total [MiB], memory.free [MiB]
0, Tesla T4, 15360 MiB, 14913 MiB
1, Tesla T4, 15360 MiB, 14913 MiB


## Step 3: Setup Environment

Installs llamatelemetry v0.1.0 with multi-GPU support and dependencies for deploying large quantized models on dual T4 GPUs.

Displays quantization guide for large models showing which parameter sizes fit on dual T4 with different quantization types (IQ3_XS, Q4_K_M).

In [3]:
%%time
print("📦 Installing dependencies...")

!pip install -q huggingface-hub sseclient-py

# Install llamatelemetry v0.1.0
!pip install -q --no-cache-dir git+https://github.com/llamatelemetry/llamatelemetry.git@v0.1.0

# Install cuGraph (matching Kaggle RAPIDS 25.6.0)
!pip install -q --extra-index-url=https://pypi.nvidia.com "cugraph-cu12==25.6.*"

# Install Graphistry
!pip install -q "graphistry[ai]" huggingface_hub

# Verify
import llamatelemetry
print(f"\n✅ llamatelemetry {llamatelemetry.__version__} installed")

try:
    import cudf, cugraph
    print(f"✅ cuDF {cudf.__version__}")
    print(f"✅ cuGraph {cugraph.__version__}")
except ImportError as e:
    print(f"⚠️ RAPIDS: {e}")

try:
    import graphistry
    print(f"✅ Graphistry {graphistry.__version__}")
except ImportError as e:
    print(f"⚠️ Graphistry: {e}")

📦 Installing dependencies...
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 24.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 MB 45.9 MB/s eta 0:00:00:00:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-adk 1.22.1 requires google-cloud-bigquery-storage>=2.0.0, which is not installed.
bigframes 2.26.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
datasets 4.4.2 requires pyarrow>=21.0.0, but you have pyarrow 19.0.1 which is incompatible.
bigframes 2.26.0 requires rich<14,>=12.4.4, but you have rich 14.2.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 332.3/332.3 kB 8.7 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━


🎯 llamatelemetry v0.1.0 First-Time Setup - Kaggle 2× T4 Multi-GPU

🎮 GPU Detected: Tesla T4 (Compute 7.5)
  ✅ Tesla T4 detected - Perfect for llamatelemetry v0.1.0!
🌐 Platform: Colab

📦 Downloading Kaggle 2× T4 binaries (~961 MB)...
    Features: FlashAttention + Tensor Cores + Multi-GPU tensor-split

➡️  Attempt 1: HuggingFace (llamatelemetry-v0.1.0-cuda12-kaggle-t4x2.tar.gz)
📥 Downloading v0.1.0 from HuggingFace Hub...
   Repo: waqasm86/llamatelemetry-binaries
   File: v0.1.0/llamatelemetry-v0.1.0-cuda12-kaggle-t4x2.tar.gz


/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:979: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(


v0.1.0/llamatelemetry-v0.1.0-cuda12-kagg(…):   0%|          | 0.00/1.40G [00:00<?, ?B/s]

🔐 Verifying SHA256 checksum...
   ✅ Checksum verified
📦 Extracting llamatelemetry-v0.1.0-cuda12-kaggle-t4x2.tar.gz...
Found 21 files in archive
Extracted 21 files to /root/.cache/llamatelemetry/extract_0.1.0
✅ Extraction complete!
  Found bin/ and lib/ under /root/.cache/llamatelemetry/extract_0.1.0/llamatelemetry-v0.1.0-cuda12-kaggle-t4x2
  Copied 13 binaries to /usr/local/lib/python3.12/dist-packages/llamatelemetry/binaries/cuda12
  Copied 2 libraries to /usr/local/lib/python3.12/dist-packages/llamatelemetry/lib
✅ Binaries installed successfully!


✅ llamatelemetry 0.1.0 installed
✅ cuDF 25.06.00
✅ cuGraph 25.06.00
✅ Graphistry 0.50.6
CPU times: user 59.8 s, sys: 14.3 s, total: 1min 14s
Wall time: 1min 53s


Calculates VRAM requirements for various large models demonstrating feasibility of 70B models with IQ3_XS quantization on dual T4.

Downloads large language model (13B+ parameters) in aggressive quantization format optimized for dual T4 VRAM constraints.

In [4]:
# Step 3: Setup Secrets
from kaggle_secrets import UserSecretsClient
from huggingface_hub import login
import os

# Setup HuggingFace token (optional for private models)
try:
    user_secrets = UserSecretsClient()
    hf_token = user_secrets.get_secret("HF_TOKEN_2")
    if hf_token:
        login(hf_token)
        print("✅ HuggingFace token loaded")
except:
    print("ℹ️ No HuggingFace token - using public models only")

# Setup Graphistry
try:
    graphistry.register(
        api=3,
        protocol="https",
        server="hub.graphistry.com",
        personal_key_id=user_secrets.get_secret("Graphistry_Personal_Key_ID"),
        personal_key_secret=user_secrets.get_secret("Graphistry_Personal_Secret_Key")
    )
    print("✅ Graphistry registered successfully")
except:
    print("ℹ️ Graphistry not registered - visualization will be limited")

✅ HuggingFace token loaded
✅ Graphistry registered successfully


## Step 4: Download 70B IQ3_XS Model

Downloads a large model (13B-70B parameters) in aggressive quantization (IQ3_XS/Q4_K_M) optimized for dual T4 VRAM constraints.

Configures multi-GPU settings with tensor-split distributing model layers equally (0.5,0.5) across both T4 GPUs for maximum capacity.

In [5]:
# Step 4: Download 3 LLM Models (1-4GB each)
from huggingface_hub import hf_hub_download
import os

print("="*70)
print("📥 DOWNLOADING 3 LLM MODELS (1-4GB EACH)")
print("="*70)

# Create models directory
os.makedirs("/kaggle/working/models", exist_ok=True)

# Model configurations - all under 4GB each
model_configs = [
    {
        "name": "Llama-3.2-3B-Instruct",
        "repo": "bartowski/Llama-3.2-3B-Instruct-GGUF",
        "file": "Llama-3.2-3B-Instruct-Q4_K_M.gguf",  # ~1.88GB
        "size_gb": 1.88
    },
    {
        "name": "Mistral-7B-Instruct",
        "repo": "TheBloke/Mistral-7B-Instruct-v0.2-GGUF",
        "file": "mistral-7b-instruct-v0.2.Q4_K_M.gguf",  # ~4.08GB
        "size_gb": 4.08
    },
    {
        "name": "Phi-3-mini-4k",
        "repo": "microsoft/Phi-3-mini-4k-instruct-gguf",
        "file": "Phi-3-mini-4k-instruct-q4.gguf",  # ~2.1GB
        "size_gb": 2.1
    }
]

model_paths = {}

for model in model_configs:
    print(f"\n📥 Downloading {model['name']}...")
    print(f"   Quantization: Q4_K_M")
    print(f"   Estimated size: {model['size_gb']:.2f} GB")
    
    try:
        # Download with timeout and progress
        model_path = hf_hub_download(
            repo_id=model["repo"],
            filename=model["file"],
            local_dir="/kaggle/working/models",
            local_dir_use_symlinks=False
        )
        
        actual_size = os.path.getsize(model_path) / (1024**3)
        model_paths[model["name"]] = model_path
        print(f"✅ Downloaded: {os.path.basename(model_path)}")
        print(f"   Actual size: {actual_size:.2f} GB")
        print(f"   Path: {model_path}")
        
    except Exception as e:
        print(f"⚠️ Failed to download {model['name']}: {e}")
        # Try alternative
        print("   Trying alternative model...")

print(f"\n📊 Summary: Downloaded {len(model_paths)} models")
for name, path in model_paths.items():
    print(f"   • {name}: {os.path.basename(path)}")

📥 DOWNLOADING 3 LLM MODELS (1-4GB EACH)

📥 Downloading Llama-3.2-3B-Instruct...
   Quantization: Q4_K_M
   Estimated size: 1.88 GB


/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:979: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(


Llama-3.2-3B-Instruct-Q4_K_M.gguf:   0%|          | 0.00/2.02G [00:00<?, ?B/s]

✅ Downloaded: Llama-3.2-3B-Instruct-Q4_K_M.gguf
   Actual size: 1.88 GB
   Path: /kaggle/working/models/Llama-3.2-3B-Instruct-Q4_K_M.gguf

📥 Downloading Mistral-7B-Instruct...
   Quantization: Q4_K_M
   Estimated size: 4.08 GB


mistral-7b-instruct-v0.2.Q4_K_M.gguf:   0%|          | 0.00/4.37G [00:00<?, ?B/s]

✅ Downloaded: mistral-7b-instruct-v0.2.Q4_K_M.gguf
   Actual size: 4.07 GB
   Path: /kaggle/working/models/mistral-7b-instruct-v0.2.Q4_K_M.gguf

📥 Downloading Phi-3-mini-4k...
   Quantization: Q4_K_M
   Estimated size: 2.10 GB


Phi-3-mini-4k-instruct-q4.gguf:   0%|          | 0.00/2.39G [00:00<?, ?B/s]

✅ Downloaded: Phi-3-mini-4k-instruct-q4.gguf
   Actual size: 2.23 GB
   Path: /kaggle/working/models/Phi-3-mini-4k-instruct-q4.gguf

📊 Summary: Downloaded 3 models
   • Llama-3.2-3B-Instruct: Llama-3.2-3B-Instruct-Q4_K_M.gguf
   • Mistral-7B-Instruct: mistral-7b-instruct-v0.2.Q4_K_M.gguf
   • Phi-3-mini-4k: Phi-3-mini-4k-instruct-q4.gguf


## Step 5: Calculate Optimal Tensor Split

Starts llama-server with dual-GPU configuration enabling models too large for single T4 to run by splitting across both GPUs.

In [6]:
# Step 5: Check GPU Availability
import subprocess

print("="*70)
print("🔍 GPU ENVIRONMENT CHECK")
print("="*70)

result = subprocess.run(
    ["nvidia-smi", "--query-gpu=index,name,memory.total,memory.free", "--format=csv,noheader"],
    capture_output=True, text=True
)

gpus = result.stdout.strip().split('\n')
print(f"\n📊 Detected {len(gpus)} GPU(s):")
for gpu in gpus:
    print(f"   {gpu}")

if len(gpus) >= 2:
    print("\n✅ Dual T4 ready for multi-model testing!")
    print("   Can run different models on different GPUs")
else:
    print("\n⚠️ Only 1 GPU detected")

🔍 GPU ENVIRONMENT CHECK

📊 Detected 2 GPU(s):
   0, Tesla T4, 15360 MiB, 14910 MiB
   1, Tesla T4, 15360 MiB, 14910 MiB

✅ Dual T4 ready for multi-model testing!
   Can run different models on different GPUs


## Step 6: Start Server with llm Models

Starts llama-server with dual-GPU tensor splitting, enabling large models that wouldn't fit on single T4 to run across both GPUs.

Verifies model distribution by checking VRAM usage on both GPUs confirming weights are properly split using tensor parallelism.

In [15]:
# Step 6: Start Single llama-server on GPU 0 (FIXED VERSION)
from llamatelemetry.server import ServerManager
import time  # Import at the top

print("="*70)
print("🚀 STARTING SINGLE LLAMA-SERVER ON GPU 0")
print("="*70)

# Choose one model to run
selected_model = None
model_name = ""

if "Llama-3.2-3B-Instruct" in model_paths:
    selected_model = model_paths["Llama-3.2-3B-Instruct"]
    model_name = "Llama-3.2-3B-Instruct"
elif "Mistral-7B-Instruct" in model_paths:
    selected_model = model_paths["Mistral-7B-Instruct"]
    model_name = "Mistral-7B-Instruct"
elif "Phi-3-mini-4k" in model_paths:
    selected_model = model_paths["Phi-3-mini-4k"]
    model_name = "Phi-3-mini-4k"

if not selected_model:
    print("❌ No models available to run!")
    # Use first available model
    for name, path in model_paths.items():
        selected_model = path
        model_name = name
        break

print(f"📋 Configuration:")
print(f"   Model: {model_name}")
print(f"   Path: {os.path.basename(selected_model)}")
print(f"   GPU 0: 100% (llama-server)")
print(f"   GPU 1: 0% (reserved for Graphistry)")

server = ServerManager()

# First try: Simple configuration based on Notebook 07
try:
    print("\n🔄 Attempt 1: Simple configuration (no flash attention)...")
    server.start_server(
        model_path=selected_model,
        host="127.0.0.1",
        port=8090,
        gpu_layers=99,
        tensor_split="1.0,0.0",  # All on GPU 0
        ctx_size=4096,
        # Skip problematic parameters
    )
    
    print("⏳ Waiting for server to start...")
    time.sleep(5)
    
    if server.check_server_health(timeout=60):
        print("✅ llama-server running on GPU 0!")
        print(f"   Server URL: http://127.0.0.1:8090")
    else:
        print("❌ Server health check failed")
        
except Exception as e:
    print(f"❌ Attempt 1 failed: {e}")
    
    # Second try: Even simpler configuration
    print("\n🔄 Attempt 2: Minimal configuration...")
    try:
        # Stop any existing server first
        try:
            server.stop_server()
        except:
            pass
        
        time.sleep(2)
        
        # Minimal parameters only
        server.start_server(
            model_path=selected_model,
            host="127.0.0.1",
            port=8090,
            gpu_layers=32,  # Reduced layers
            tensor_split="1.0,0.0",
            ctx_size=2048,  # Reduced context
        )
        
        time.sleep(5)
        if server.check_server_health(timeout=60):
            print("✅ Server started with minimal config!")
        else:
            print("❌ Server health check failed")
            
    except Exception as e2:
        print(f"❌ Attempt 2 failed: {e2}")
        
        # Third try: Use parameters from working Notebook 07
        print("\n🔄 Attempt 3: Copy configuration from Notebook 07...")
        try:
            try:
                server.stop_server()
            except:
                pass
            
            time.sleep(2)
            
            # Exact parameters from Notebook 07 (which works)
            server.start_server(
                model_path=selected_model,
                host="127.0.0.1",
                port=8090,
                gpu_layers=99,
                tensor_split="1.0,0.0",
                ctx_size=4096,
                # No additional parameters
            )
            
            time.sleep(5)
            if server.check_server_health(timeout=60):
                print("✅ Server started with Notebook 07 config!")
            else:
                print("❌ Still failing")
                print("\n🔧 Debug: Checking server process...")
                import subprocess
                result = subprocess.run(["ps", "aux"], capture_output=True, text=True)
                if "llama-server" in result.stdout:
                    print("⚠️ llama-server process found but not responding")
                else:
                    print("❌ No llama-server process found")
                    
        except Exception as e3:
            print(f"❌ All attempts failed: {e3}")
            print("\n🔧 Troubleshooting steps:")
            print("1. Check if model file exists and is valid")
            print(f"   Path: {selected_model}")
            print(f"   Exists: {os.path.exists(selected_model)}")
            print(f"   Size: {os.path.getsize(selected_model) / 1024**3:.2f} GB")
            
            print("\n2. Try manual start:")
            print(f"   !/usr/local/lib/python3.12/dist-packages/llamatelemetry/binaries/cuda12/llama-server \\")
            print(f"     --model {selected_model} \\")
            print(f"     --host 127.0.0.1 \\")
            print(f"     --port 8090 \\")
            print(f"     --n-gpu-layers 99 \\")
            print(f"     --ctx-size 4096 \\")
            print(f"     --tensor-split 1.0,0.0")

🚀 STARTING SINGLE LLAMA-SERVER ON GPU 0
📋 Configuration:
   Model: Llama-3.2-3B-Instruct
   Path: Llama-3.2-3B-Instruct-Q4_K_M.gguf
   GPU 0: 100% (llama-server)
   GPU 1: 0% (reserved for Graphistry)

🔄 Attempt 1: Simple configuration (no flash attention)...
GPU Check:
  Platform: kaggle
  GPU: Tesla T4
  Compute Capability: 7.5
  Status: ✓ Compatible
Starting llama-server...
  Executable: /usr/local/lib/python3.12/dist-packages/llamatelemetry/binaries/cuda12/llama-server
  Model: Llama-3.2-3B-Instruct-Q4_K_M.gguf
  GPU Layers: 99
  Context Size: 4096
  Server URL: http://127.0.0.1:8090
Waiting for server to be ready....... ✓ Ready in 4.1s
⏳ Waiting for server to start...
✅ llama-server running on GPU 0!
   Server URL: http://127.0.0.1:8090


In [7]:
# Step 6: Start Multiple llama-servers for All Three Models
from llamatelemetry.server import ServerManager
import time
import threading
import subprocess

print("="*70)
print("🚀 STARTING MULTIPLE LLAMA-SERVERS FOR ALL MODELS")
print("="*70)

# Define server configurations for each model
servers_config = []
if model_paths:
    # Use different ports for each model
    ports = [8090, 8091, 8092]
    gpu_assignments = ["1.0,0.0", "0.5,0.5", "0.0,1.0"]  # Different GPU allocations
    
    for i, (model_name, model_path) in enumerate(model_paths.items()):
        if i < len(ports):
            servers_config.append({
                "name": model_name,
                "path": model_path,
                "port": ports[i],
                "gpu_assignment": gpu_assignments[i] if len(gpu_assignments) > i else "1.0,0.0"
            })

print("📋 Configuration:")
for config in servers_config:
    print(f"   Model: {config['name']}")
    print(f"   Port: {config['port']}")
    print(f"   GPU Split: {config['gpu_assignment']}")
    print(f"   Path: {os.path.basename(config['path'])}")
    print()

# Function to start a single server
def start_single_server(config):
    server = ServerManager()
    
    print(f"\n🔄 Starting server for {config['name']} on port {config['port']}...")
    
    try:
        server.start_server(
            model_path=config["path"],
            host="127.0.0.1",
            port=config["port"],
            gpu_layers=99,
            tensor_split=config["gpu_assignment"],
            ctx_size=4096,
        )
        
        time.sleep(3)
        
        if server.check_server_health(timeout=30):
            print(f"✅ {config['name']} server running on port {config['port']}!")
            return server
        else:
            print(f"❌ {config['name']} server health check failed")
            return None
            
    except Exception as e:
        print(f"❌ Failed to start {config['name']} server: {e}")
        
        # Try with simpler config
        try:
            server.stop_server()
            time.sleep(2)
            
            server.start_server(
                model_path=config["path"],
                host="127.0.0.1",
                port=config["port"],
                gpu_layers=32,
                tensor_split=config["gpu_assignment"],
                ctx_size=2048,
            )
            
            time.sleep(3)
            if server.check_server_health(timeout=30):
                print(f"✅ {config['name']} server started with minimal config!")
                return server
        except Exception as e2:
            print(f"❌ {config['name']} server failed completely: {e2}")
            return None

# Start all servers
active_servers = {}
for config in servers_config:
    server = start_single_server(config)
    if server:
        active_servers[config["name"]] = {
            "server": server,
            "port": config["port"],
            "gpu_assignment": config["gpu_assignment"]
        }

print(f"\n📊 Summary: {len(active_servers)} servers running")
for name, info in active_servers.items():
    print(f"   • {name}: http://127.0.0.1:{info['port']} (GPU: {info['gpu_assignment']})")

🚀 STARTING MULTIPLE LLAMA-SERVERS FOR ALL MODELS
📋 Configuration:
   Model: Llama-3.2-3B-Instruct
   Port: 8090
   GPU Split: 1.0,0.0
   Path: Llama-3.2-3B-Instruct-Q4_K_M.gguf

   Model: Mistral-7B-Instruct
   Port: 8091
   GPU Split: 0.5,0.5
   Path: mistral-7b-instruct-v0.2.Q4_K_M.gguf

   Model: Phi-3-mini-4k
   Port: 8092
   GPU Split: 0.0,1.0
   Path: Phi-3-mini-4k-instruct-q4.gguf


🔄 Starting server for Llama-3.2-3B-Instruct on port 8090...
GPU Check:
  Platform: kaggle
  GPU: Tesla T4
  Compute Capability: 7.5
  Status: ✓ Compatible
Starting llama-server...
  Executable: /usr/local/lib/python3.12/dist-packages/llamatelemetry/binaries/cuda12/llama-server
  Model: Llama-3.2-3B-Instruct-Q4_K_M.gguf
  GPU Layers: 99
  Context Size: 4096
  Server URL: http://127.0.0.1:8090
Waiting for server to be ready...... ✓ Ready in 3.1s
✅ Llama-3.2-3B-Instruct server running on port 8090!

🔄 Starting server for Mistral-7B-Instruct on port 8091...
GPU Check:
  Platform: kaggle
  GPU: Tesla T4
 

## Step 7: Verify GPU Memory Usage

Benchmarks large model inference performance measuring tokens/second, latency, and throughput on dual-GPU setup.

In [16]:
print("="*70)
print("💾 GPU MEMORY USAGE")
print("="*70)

!nvidia-smi

print("\n📊 Expected memory distribution:")
print("   GPU 0: ~12-13 GB (model layers 0-39)")
print("   GPU 1: ~12-13 GB (model layers 40-79)")

💾 GPU MEMORY USAGE
Thu Feb  5 03:06:08 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 580.105.08             Driver Version: 580.105.08     CUDA Version: 13.0     |
+-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   50C    P0             27W /   70W |    2565MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+----------------------------

In [8]:
print("="*70)
print("💾 GPU MEMORY USAGE - ALL MODELS")
print("="*70)

# Check memory usage
!nvidia-smi

print("\n📊 Expected memory distribution:")
print("   GPU 0: Llama-3.2-3B-Instruct (100%)")
print("   GPU 0: Mistral-7B-Instruct (50%) + GPU 1: Mistral-7B-Instruct (50%)")
print("   GPU 1: Phi-3-mini-4k (100%)")

💾 GPU MEMORY USAGE - ALL MODELS
Thu Feb  5 04:00:24 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 580.105.08             Driver Version: 580.105.08     CUDA Version: 13.0     |
+-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   62C    P0             31W /   70W |    2565MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+---------------

## Step 8: Test Inference

Tests different context window sizes (2K, 4K, 8K) to find optimal balance between context length and available VRAM.

In [17]:
from llamatelemetry.api.client import LlamaCppClient
import time

print("="*70)
print("="*70)

client = LlamaCppClient(base_url="http://127.0.0.1:8090")

# Test prompt
prompt = "Explain quantum computing in simple terms."

print(f"\n💬 Prompt: {prompt}")
print("\n📝 Response:\n")

start = time.time()

response = client.chat.create(
    messages=[{"role": "user", "content": prompt}],
    max_tokens=200,
    temperature=0.7,
)

elapsed = time.time() - start
tokens = response.usage.completion_tokens

print(response.choices[0].message.content)

print(f"\n📊 Performance:")
print(f"   Time: {elapsed:.2f}s")
print(f"   Tokens: {tokens}")
print(f"   Speed: {tokens/elapsed:.1f} tok/s")


💬 Prompt: Explain quantum computing in simple terms.

📝 Response:

Quantum computing is a new way of processing information that's different from the traditional computers we use every day.

**Traditional Computers**

In a traditional computer, information is represented as 0s and 1s. These 0s and 1s are stored on tiny switches called transistors inside the computer's brain (called the CPU). The computer processes information by flipping these switches between 0 and 1.

**Quantum Computers**

But in a quantum computer, information is represented as special particles called qubits (quantum bits). These qubits can exist as both 0 AND 1 at the same time, which means they can process multiple possibilities simultaneously. This is different from traditional computers, where information is either 0 or 1.

Think of it like a coin flip. In a traditional computer, you'd flip a coin and get heads (0) or tails (1). But in a quantum computer, the coin could be both heads AND tails at the same tim

In [9]:
from llamatelemetry.api.client import LlamaCppClient
import time
import pandas as pd

print("="*70)
print("🤖 TEST INFERENCE - ALL MODELS")
print("="*70)

# Test prompts
test_prompts = [
    "Explain quantum computing in simple terms.",
    "What is the capital of France?",
    "Write a Python function to calculate factorial."
]

# Create clients for each server
clients = {}
for name, info in active_servers.items():
    clients[name] = LlamaCppClient(base_url=f"http://127.0.0.1:{info['port']}")

# Test each model with each prompt
results = []

for model_name, client in clients.items():
    print(f"\n🧪 Testing {model_name}:")
    print("-" * 40)
    
    for prompt in test_prompts[:1]:  # Test with first prompt for brevity
        print(f"\n💬 Prompt: {prompt}")
        
        start = time.time()
        try:
            response = client.chat.create(
                messages=[{"role": "user", "content": prompt}],
                max_tokens=100,
                temperature=0.7,
            )
            
            elapsed = time.time() - start
            tokens = response.usage.completion_tokens
            
            # Display first 200 chars of response
            response_text = response.choices[0].message.content
            print(f"📝 Response (first 200 chars): {response_text[:200]}...")
            
            print(f"📊 Performance: {elapsed:.2f}s | {tokens} tokens | {tokens/elapsed:.1f} tok/s")
            
            results.append({
                "Model": model_name,
                "Prompt": prompt[:30] + "...",
                "Time (s)": f"{elapsed:.2f}",
                "Tokens": tokens,
                "Speed (tok/s)": f"{tokens/elapsed:.1f}"
            })
            
        except Exception as e:
            print(f"❌ Error: {e}")
            results.append({
                "Model": model_name,
                "Prompt": prompt[:30] + "...",
                "Time (s)": "Error",
                "Tokens": 0,
                "Speed (tok/s)": "N/A"
            })

# Display results as a table
print("\n" + "="*70)
print("📋 INFERENCE RESULTS SUMMARY")
print("="*70)

if results:
    df = pd.DataFrame(results)
    print(df.to_string(index=False))
else:
    print("No results collected")

🤖 TEST INFERENCE - ALL MODELS

🧪 Testing Llama-3.2-3B-Instruct:
----------------------------------------

💬 Prompt: Explain quantum computing in simple terms.
📝 Response (first 200 chars): Imagine you have a huge library with an infinite number of books, and each book represents a possible solution to a problem. A regular computer would look through the books one by one, checking each s...
📊 Performance: 1.62s | 100 tokens | 61.8 tok/s

🧪 Testing Mistral-7B-Instruct:
----------------------------------------

💬 Prompt: Explain quantum computing in simple terms.
❌ Error: HTTPConnectionPool(host='127.0.0.1', port=8091): Max retries exceeded with url: /v1/chat/completions (Caused by NewConnectionError("HTTPConnection(host='127.0.0.1', port=8091): Failed to establish a new connection: [Errno 111] Connection refused"))

🧪 Testing Phi-3-mini-4k:
----------------------------------------

💬 Prompt: Explain quantum computing in simple terms.
❌ Error: HTTPConnectionPool(host='127.0.0.1', port=809

## Step 9: Performance Benchmarks

Benchmarks large model inference performance measuring tokens/second throughput on the dual-GPU setup.

Compares quantization quality by testing same model in IQ3_XS vs Q4_K_M showing quality/size trade-offs for large models.

In [18]:
import time

print("="*70)
print("📊 PERFORMANCE BENCHMARKS")
print("="*70)

prompts = [
    "Hello",  # Very short
    "What is the capital of France?",  # Short
    "Write a paragraph about machine learning.",  # Medium
    "Explain the differences between Python and JavaScript with examples.",  # Long
]

print("\n🏃 Running benchmarks...\n")

for prompt in prompts:
    start = time.time()
    
    response = client.chat.create(
        messages=[{"role": "user", "content": prompt}],
        max_tokens=100,
        temperature=0.7,
    )
    
    elapsed = time.time() - start
    tokens = response.usage.completion_tokens
    
    print(f"   Prompt: \"{prompt[:40]}...\"")
    print(f"   Time: {elapsed:.2f}s | Tokens: {tokens} | Speed: {tokens/elapsed:.1f} tok/s")
    print()

print("""
📝 Expected Performance (70B on Dual T4):
   • IQ3_XS: ~5-10 tok/s
   • IQ2_XXS: ~8-15 tok/s (lower quality)
   
   Compared to:
   • 7B Q4_K_M on single T4: ~40-60 tok/s
   • 13B Q4_K_M on single T4: ~25-35 tok/s
""")

📊 PERFORMANCE BENCHMARKS

🏃 Running benchmarks...

   Prompt: "Hello..."
   Time: 0.21s | Tokens: 8 | Speed: 37.3 tok/s

   Prompt: "What is the capital of France?..."
   Time: 0.12s | Tokens: 8 | Speed: 66.2 tok/s

   Prompt: "Write a paragraph about machine learning..."
   Time: 1.39s | Tokens: 100 | Speed: 72.0 tok/s

   Prompt: "Explain the differences between Python a..."
   Time: 1.41s | Tokens: 100 | Speed: 71.1 tok/s


📝 Expected Performance (70B on Dual T4):
   • IQ3_XS: ~5-10 tok/s
   • IQ2_XXS: ~8-15 tok/s (lower quality)
   
   Compared to:
   • 7B Q4_K_M on single T4: ~40-60 tok/s
   • 13B Q4_K_M on single T4: ~25-35 tok/s



In [10]:
import time

print("="*70)
print("📊 PERFORMANCE BENCHMARKS - ALL MODELS")
print("="*70)

# Define benchmark prompts
benchmark_prompts = [
    "Hello, how are you?",
    "Explain the concept of machine learning.",
    "Write a short story about a robot learning to paint.",
    "What are the main differences between Python and JavaScript?"
]

print("\n🏃 Running benchmarks for all models...\n")

benchmark_results = []

for model_name, client in clients.items():
    print(f"\n🔬 Benchmarking {model_name}:")
    print("-" * 40)
    
    model_times = []
    model_speeds = []
    
    for i, prompt in enumerate(benchmark_prompts):
        start = time.time()
        
        try:
            response = client.chat.create(
                messages=[{"role": "user", "content": prompt}],
                max_tokens=50,  # Fixed tokens for fair comparison
                temperature=0.7,
            )
            
            elapsed = time.time() - start
            tokens = response.usage.completion_tokens
            
            speed = tokens / elapsed if elapsed > 0 else 0
            
            print(f"   Prompt {i+1}: {elapsed:.2f}s | {speed:.1f} tok/s")
            
            model_times.append(elapsed)
            model_speeds.append(speed)
            
        except Exception as e:
            print(f"   Prompt {i+1}: Error - {e}")
            model_times.append(0)
            model_speeds.append(0)
    
    # Calculate averages
    avg_time = sum(model_times) / len(model_times) if model_times else 0
    avg_speed = sum(model_speeds) / len(model_speeds) if model_speeds else 0
    
    benchmark_results.append({
        "Model": model_name,
        "Avg Time (s)": f"{avg_time:.2f}",
        "Avg Speed (tok/s)": f"{avg_speed:.1f}",
        "GPU Assignment": active_servers[model_name]["gpu_assignment"]
    })
    
    print(f"   📊 Average: {avg_time:.2f}s | {avg_speed:.1f} tok/s")

# Display benchmark summary
print("\n" + "="*70)
print("🏆 BENCHMARK SUMMARY")
print("="*70)

if benchmark_results:
    df_benchmark = pd.DataFrame(benchmark_results)
    print(df_benchmark.to_string(index=False))
    
    # Find fastest model
    fastest = max(benchmark_results, key=lambda x: float(x["Avg Speed (tok/s)"]))
    print(f"\n🎯 Fastest Model: {fastest['Model']} ({fastest['Avg Speed (tok/s)']} tok/s)")
    
    # Compare model sizes
    print("\n📈 Model Size Comparison:")
    model_sizes = {
        "Llama-3.2-3B-Instruct": "3B",
        "Mistral-7B-Instruct": "7B", 
        "Phi-3-mini-4k": "3.8B"
    }
    
    for model_name in model_paths.keys():
        if model_name in model_sizes:
            size = model_sizes[model_name]
            speed = next((r["Avg Speed (tok/s)"] for r in benchmark_results if r["Model"] == model_name), "N/A")
            print(f"   • {model_name} ({size}): {speed} tok/s")

📊 PERFORMANCE BENCHMARKS - ALL MODELS

🏃 Running benchmarks for all models...


🔬 Benchmarking Llama-3.2-3B-Instruct:
----------------------------------------
   Prompt 1: 0.80s | 58.4 tok/s
   Prompt 2: 0.81s | 61.7 tok/s
   Prompt 3: 0.80s | 62.8 tok/s
   Prompt 4: 0.81s | 61.7 tok/s
   📊 Average: 0.81s | 61.1 tok/s

🔬 Benchmarking Mistral-7B-Instruct:
----------------------------------------
   Prompt 1: Error - HTTPConnectionPool(host='127.0.0.1', port=8091): Max retries exceeded with url: /v1/chat/completions (Caused by NewConnectionError("HTTPConnection(host='127.0.0.1', port=8091): Failed to establish a new connection: [Errno 111] Connection refused"))
   Prompt 2: Error - HTTPConnectionPool(host='127.0.0.1', port=8091): Max retries exceeded with url: /v1/chat/completions (Caused by NewConnectionError("HTTPConnection(host='127.0.0.1', port=8091): Failed to establish a new connection: [Errno 111] Connection refused"))
   Prompt 3: Error - HTTPConnectionPool(host='127.0.0.1', port

## Step 9B: Model Performance Landscape Visualization

Demonstrates batch processing optimizations for large models including optimal batch sizes and parallel request handling.

In [19]:
print("="*70)
print("🎨 MODEL PERFORMANCE LANDSCAPE - GRAPHISTRY VISUALIZATION")
print("="*70)

import graphistry
from kaggle_secrets import UserSecretsClient
import pandas as pd
import numpy as np

# --- 1. Register Graphistry ---
print("\n🔐 Registering with Graphistry...")
try:
    user_secrets = UserSecretsClient()
    graphistry_key_id = user_secrets.get_secret("Graphistry_Personal_Key_ID")
    graphistry_secret = user_secrets.get_secret("Graphistry_Personal_Secret_Key")

    graphistry.register(
        api=3,
        protocol="https",
        server="hub.graphistry.com",
        personal_key_id=graphistry_key_id,
        personal_key_secret=graphistry_secret
    )
    print("✅ Graphistry registered successfully")
except Exception as e:
    print(f"⚠️ Graphistry registration failed: {e}")
    print("   Add secrets: Graphistry_Personal_Key_ID, Graphistry_Personal_Secret_Key")

# --- 2. Build Model Performance Network ---
print("\n📊 Building model performance landscape...")

# Model configurations with performance metrics
models_data = [
    # Model Size, Quantization, VRAM, Speed (tok/s), Quality (stars), Fits T4, GPU Mode
    {"model": "1B-3B", "quant": "Q4_K_M", "vram_gb": 2.5, "speed": 80, "quality": 3, "fits": "Single", "category": "Small"},
    {"model": "7B", "quant": "Q4_K_M", "vram_gb": 5.5, "speed": 55, "quality": 4, "fits": "Single", "category": "Medium"},
    {"model": "7B", "quant": "F16", "vram_gb": 15, "speed": 45, "quality": 5, "fits": "Single", "category": "Medium"},
    {"model": "13B", "quant": "Q4_K_M", "vram_gb": 8.5, "speed": 32, "quality": 4, "fits": "Single", "category": "Medium"},
    {"model": "13B", "quant": "Q8_0", "vram_gb": 14.5, "speed": 28, "quality": 5, "fits": "Single", "category": "Medium"},
    {"model": "32B", "quant": "Q4_K_M", "vram_gb": 21, "speed": 20, "quality": 5, "fits": "Dual", "category": "Large"},
    {"model": "70B", "quant": "IQ4_XS", "vram_gb": 35, "speed": 6, "quality": 4, "fits": "None", "category": "XLarge"},
    {"model": "70B", "quant": "IQ3_M", "vram_gb": 28, "speed": 7, "quality": 4, "fits": "Dual", "category": "XLarge"},
    {"model": "70B", "quant": "IQ3_XS", "vram_gb": 25, "speed": 8, "quality": 3, "fits": "Dual", "category": "XLarge"},
    {"model": "70B", "quant": "IQ2_XXS", "vram_gb": 18, "speed": 12, "quality": 2, "fits": "Dual", "category": "XLarge"},
    {"model": "70B", "quant": "IQ2_XS", "vram_gb": 20, "speed": 11, "quality": 2, "fits": "Dual", "category": "XLarge"},
    {"model": "70B", "quant": "IQ1_M", "vram_gb": 16, "speed": 15, "quality": 1, "fits": "Dual", "category": "XLarge"},
]

# Create nodes DataFrame
nodes_df = pd.DataFrame(models_data)
nodes_df['node_id'] = range(len(nodes_df))
nodes_df['label'] = nodes_df['model'] + "\\n" + nodes_df['quant']

# --- 3. Create Compatibility Edges ---
# Connect models that can work together or represent upgrade paths
edges_list = []

for i, model1 in nodes_df.iterrows():
    for j, model2 in nodes_df.iterrows():
        if i >= j:
            continue
        
        # Same model size, different quantization = quantization tradeoff edge
        if model1['model'] == model2['model']:
            edges_list.append({
                'source': i,
                'target': j,
                'relationship': 'quant_tradeoff',
                'weight': abs(model1['quality'] - model2['quality']) + 1,
                'edge_label': 'Quant Options'
            })
        
        # Compatible VRAM range = viable upgrade path
        elif (abs(model1['vram_gb'] - model2['vram_gb']) < 8 and 
              model1['category'] != model2['category']):
            edges_list.append({
                'source': i,
                'target': j,
                'relationship': 'upgrade_path',
                'weight': abs(model1['speed'] - model2['speed']) / 10 + 1,
                'edge_label': 'Upgrade Path'
            })

edges_df = pd.DataFrame(edges_list)

print(f"   Models: {len(nodes_df)}")
print(f"   Relationships: {len(edges_df)}")

# --- 4. Enhanced Node Attributes ---

# Compute derived metrics
nodes_df['efficiency'] = nodes_df['speed'] / nodes_df['vram_gb']  # tok/s per GB
nodes_df['value_score'] = (nodes_df['speed'] * nodes_df['quality']) / (nodes_df['vram_gb'] / 10)

# Size encoding (based on value score)
vmin, vmax = nodes_df['value_score'].min(), nodes_df['value_score'].max()
nodes_df['node_size'] = 30 + (nodes_df['value_score'] - vmin) / (vmax - vmin) * 70

# Role classification
def classify_role(row):
    if row['fits'] == 'None':
        return 'Too Large'
    elif row['speed'] > 40:
        return 'Speed King'
    elif row['quality'] >= 4:
        return 'Quality Leader'
    elif row['vram_gb'] > 20:
        return 'Large Model'
    else:
        return 'Balanced'

nodes_df['role'] = nodes_df.apply(classify_role, axis=1)

# Rich tooltips
nodes_df['point_title'] = nodes_df.apply(
    lambda row: f"{row['label'].replace(chr(92)*2+'n', ' ')}\\n" +
                f"VRAM: {row['vram_gb']:.1f} GB | Speed: {row['speed']} tok/s\\n" +
                f"Quality: {'⭐' * int(row['quality'])}\\n" +
                f"Fits: {row['fits']} GPU | Role: {row['role']}\\n" +
                f"Efficiency: {row['efficiency']:.1f} tok/s/GB\\n" +
                f"Value Score: {row['value_score']:.1f}",
    axis=1
)

edges_df['edge_title'] = edges_df.apply(
    lambda row: f"{row['relationship'].replace('_', ' ').title()}\\n" +
                f"Strength: {row['weight']:.1f}",
    axis=1
)

# --- 5. Create Graphistry Visualization ---
print("\n🎨 Creating model performance landscape...")

# Color palettes
category_colors = {
    'Small': '#4ECDC4',      # Teal - Small models
    'Medium': '#45B7D1',     # Blue - Medium models  
    'Large': '#FFA07A',      # Orange - Large models
    'XLarge': '#FF6B6B',     # Red - Extra large models
}

fits_colors = {
    'Single': '#2ECC71',     # Green - Fits single GPU
    'Dual': '#F39C12',       # Orange - Needs dual GPU
    'None': '#E74C3C'        # Red - Doesn't fit
}

role_icons = {
    'Speed King': 'rocket',
    'Quality Leader': 'trophy',
    'Large Model': 'database',
    'Balanced': 'balance-scale',
    'Too Large': 'exclamation-triangle'
}

relationship_colors = {
    'quant_tradeoff': '#9B59B6',  # Purple - Quantization options
    'upgrade_path': '#3498DB'     # Blue - Upgrade paths
}

# Bind graph
g = graphistry.bind(
    source='source',
    destination='target',
    node='node_id',
    point_title='point_title',
    edge_title='edge_title'
)

# Build visualization with advanced encodings
plotter = (
    g.edges(edges_df)
    .nodes(nodes_df)
    
    # Primary encoding: Color by model category
    .encode_point_color('category', categorical_mapping=category_colors, default_mapping='#95A5A6')
    
    # Size by value score (quality * speed / VRAM)
    .encode_point_size('node_size')
    
    # Icon by role
    .encode_point_icon('role', categorical_mapping=role_icons, default_mapping='circle')
    
    # Badge: GPU fit indicator
    .encode_point_badge(
        'fits',
        'TopRight',
        shape='circle',
        categorical_mapping={
            'Single': 'microchip',
            'Dual': 'th',
            'None': 'ban'
        },
        color={'mapping': {'categorical': {'fixed': fits_colors, 'other': '#999'}}}
    )
    
    # Edge color by relationship type
    .encode_edge_color('relationship', categorical_mapping=relationship_colors)
    
    # Settings for performance visualization
    .settings(url_params={
        'play': 2000,
        'pointSize': 2.0,
        'edgeOpacity': 0.5,
        'bg': '%23FFFFFF',
        'showArrows': 'true',
        'showLabels': 'true',
        'strongGravity': 'true',
        'edgeInfluence': 1.5,
        'pointsOfInterestMax': 5
    })
)

# --- 6. Launch Visualization ---
try:
    url = plotter.plot(
        render=False,
        name="Model Performance Landscape - llamatelemetry v0.1.0",
        description=f"Interactive comparison of {len(nodes_df)} model configurations for Kaggle Dual T4"
    )

    print(f"\n🚀 Visualization Created Successfully!")
    print(f"\n🔗 Graphistry URL:")
    print(f"   {url}")
    print(f"\n📌 Visualization Features:")
    print(f"   ✓ Color: Model category (Small/Medium/Large/XLarge)")
    print(f"   ✓ Size: Value score (speed×quality/VRAM)")
    print(f"   ✓ Icon: Performance role (Speed King, Quality Leader, etc.)")
    print(f"   ✓ Badge: GPU fit indicator (Single/Dual/None)")
    print(f"   ✓ Edges: Quantization tradeoffs & upgrade paths")
    print(f"   ✓ Tooltips: Full specs, efficiency, value scores")
    
    print(f"\n🔍 Key Insights:")
    print(f"   • Purple edges: Different quantizations of same model")
    print(f"   • Blue edges: Viable upgrade paths (similar VRAM)")
    print(f"   • Larger nodes: Better value (high speed×quality, low VRAM)")
    print(f"   • Green badge: Fits single T4 (15GB)")
    print(f"   • Orange badge: Requires dual T4 split")
    print(f"   • Red badge: Too large for dual T4")

    from IPython.display import display, HTML
    display(HTML(
        f'<div style="margin:20px; padding:20px; background:linear-gradient(135deg, #667eea 0%, #764ba2 100%); '
        f'border-radius:12px; color:white; box-shadow:0 4px 6px rgba(0,0,0,0.1);">'
        f'<h3 style="margin:0 0 10px 0;">🎯 Model Selection Dashboard</h3>'
        f'<p style="margin:5px 0;">Explore {len(nodes_df)} configurations across 4 model sizes</p>'
        f'<p style="margin:5px 0; font-size:14px;">Compare quantization tradeoffs, performance, and GPU compatibility</p>'
        f'<a href="{url}" target="_blank" style="display:inline-block; margin-top:15px; padding:12px 24px; '
        f'background:white; color:#667eea; text-decoration:none; border-radius:6px; font-weight:bold; '
        f'box-shadow:0 2px 4px rgba(0,0,0,0.1);">🚀 Open Interactive Dashboard</a>'
        f'</div>'
    ))

except Exception as e:
    print(f"\n❌ Visualization error: {e}")
    print(f"   Data prepared successfully - {len(nodes_df)} nodes, {len(edges_df)} edges")

print("\n" + "="*70)
print("✅ Model performance landscape visualization complete")
print("="*70)

🎨 MODEL PERFORMANCE LANDSCAPE - GRAPHISTRY VISUALIZATION

🔐 Registering with Graphistry...
✅ Graphistry registered successfully

📊 Building model performance landscape...
   Models: 12
   Relationships: 33

🎨 Creating model performance landscape...

🚀 Visualization Created Successfully!

🔗 Graphistry URL:
   https://hub.graphistry.com/graph/graph.html?dataset=97a8a1adee474301af5649921b1fce65&type=arrow&viztoken=8df625a6-2901-46dc-8419-8d0b7b980748&usertag=abf3123b-pygraphistry-0.50.6&splashAfter=1770260808&info=true&play=2000&pointSize=2.0&edgeOpacity=0.5&bg=%23FFFFFF&showArrows=true&showLabels=true&strongGravity=true&edgeInfluence=1.5&pointsOfInterestMax=5

📌 Visualization Features:
   ✓ Color: Model category (Small/Medium/Large/XLarge)
   ✓ Size: Value score (speed×quality/VRAM)
   ✓ Icon: Performance role (Speed King, Quality Leader, etc.)
   ✓ Badge: GPU fit indicator (Single/Dual/None)
   ✓ Edges: Quantization tradeoffs & upgrade paths
   ✓ Tooltips: Full specs, efficiency, value 


✅ Model performance landscape visualization complete


## Step 11: Streaming for Better UX

Analyzes memory usage patterns during inference showing how VRAM is consumed by model weights, KV cache, and activations.

In [20]:
print("="*70)
print("🌊 STREAMING WITH LARGE MODELS")
print("="*70)

print("\n💬 Streaming response (feels faster!):\n")

start = time.time()
token_count = 0

try:
    # Enable streaming with stream=True
    stream_response = client.chat.create(
        messages=[{"role": "user", "content": "What are the benefits of streaming?"}],
        max_tokens=100,
        temperature=0.7,
        stream=True  # This enables streaming
    )
    
    print("Response: ", end="", flush=True)
    
    # Iterate through the stream
    for chunk in stream_response:
        if hasattr(chunk, 'choices') and chunk.choices:
            choice = chunk.choices[0]
            if hasattr(choice, 'delta') and hasattr(choice.delta, 'content') and choice.delta.content:
                content = choice.delta.content
                print(content, end="", flush=True)
                token_count += 1
            elif hasattr(choice, 'message') and hasattr(choice.message, 'content') and choice.message.content:
                # Some APIs might use message instead of delta
                content = choice.message.content
                print(content, end="", flush=True)
                token_count += 1
    
    elapsed = time.time() - start
    print(f"\n\n📊 Streamed {token_count} tokens in {elapsed:.2f}s")
    if elapsed > 0:
        print(f"📈 Speed: {token_count/elapsed:.1f} tokens/sec")
    print("\n✅ Streaming provides immediate feedback even with slower models!")

except Exception as e:
    print(f"\n❌ Streaming error: {e}")
    
    # Fallback to non-streaming
    print("\n🔄 Trying non-streaming fallback...")
    try:
        start = time.time()
        response = client.chat.create(
            messages=[{"role": "user", "content": "What are the benefits of streaming?"}],
            max_tokens=100,
            temperature=0.7,
            stream=False
        )
        
        if response.choices and response.choices[0].message.content:
            content = response.choices[0].message.content
            print(f"Response: {content}")
            
            # Estimate tokens (rough approximation)
            token_count = len(content.split())
            elapsed = time.time() - start
            print(f"\n📊 Generated {token_count} tokens in {elapsed:.2f}s")
            if elapsed > 0:
                print(f"📈 Speed: {token_count/elapsed:.1f} tokens/sec")
        else:
            print("⚠️ No response content received")
            
    except Exception as e2:
        print(f"❌ Fallback also failed: {e2}")

🌊 STREAMING WITH LARGE MODELS

💬 Streaming response (feels faster!):

Response: 

📊 Streamed 0 tokens in 1.50s
📈 Speed: 0.0 tokens/sec

✅ Streaming provides immediate feedback even with slower models!


In [11]:
print("="*70)
print("🌊 STREAMING TEST - ALL MODELS")
print("="*70)

stream_prompt = "Explain the benefits of streaming in large language models."

for model_name, client in clients.items():
    print(f"\n💭 Testing streaming for {model_name}:")
    print("-" * 40)
    
    print(f"Prompt: {stream_prompt}")
    print("Response stream: ", end="", flush=True)
    
    start = time.time()
    token_count = 0
    
    try:
        stream_response = client.chat.create(
            messages=[{"role": "user", "content": stream_prompt}],
            max_tokens=80,
            temperature=0.7,
            stream=True
        )
        
        for chunk in stream_response:
            if hasattr(chunk, 'choices') and chunk.choices:
                choice = chunk.choices[0]
                if hasattr(choice, 'delta') and hasattr(choice.delta, 'content') and choice.delta.content:
                    content = choice.delta.content
                    print(content, end="", flush=True)
                    token_count += 1
        
        elapsed = time.time() - start
        print(f"\n\n📊 Streamed {token_count} tokens in {elapsed:.2f}s ({token_count/elapsed:.1f} tok/s)")
        
    except Exception as e:
        print(f"\n❌ Streaming error: {e}")
        
        # Fallback to non-streaming
        try:
            start = time.time()
            response = client.chat.create(
                messages=[{"role": "user", "content": stream_prompt}],
                max_tokens=80,
                temperature=0.7,
                stream=False
            )
            
            if response.choices and response.choices[0].message.content:
                elapsed = time.time() - start
                print(f"📝 Non-streaming response generated in {elapsed:.2f}s")
        except Exception as e2:
            print(f"❌ Fallback also failed: {e2}")

🌊 STREAMING TEST - ALL MODELS

💭 Testing streaming for Llama-3.2-3B-Instruct:
----------------------------------------
Prompt: Explain the benefits of streaming in large language models.
Response stream: 

📊 Streamed 0 tokens in 1.34s (0.0 tok/s)

💭 Testing streaming for Mistral-7B-Instruct:
----------------------------------------
Prompt: Explain the benefits of streaming in large language models.
Response stream: 
❌ Streaming error: HTTPConnectionPool(host='127.0.0.1', port=8091): Max retries exceeded with url: /v1/chat/completions (Caused by NewConnectionError("HTTPConnection(host='127.0.0.1', port=8091): Failed to establish a new connection: [Errno 111] Connection refused"))
❌ Fallback also failed: HTTPConnectionPool(host='127.0.0.1', port=8091): Max retries exceeded with url: /v1/chat/completions (Caused by NewConnectionError("HTTPConnection(host='127.0.0.1', port=8091): Failed to establish a new connection: [Errno 111] Connection refused"))

💭 Testing streaming for Phi-3-mini-4k:

## Step 12: Memory Tips

Demonstrates memory optimization techniques (batch size, context size) for maximizing large model performance on dual T4.

Tests prompt caching and KV cache optimization techniques to improve throughput for repeated or similar prompts.

In [21]:
print("="*70)
print("💡 MEMORY OPTIMIZATION TIPS")
print("="*70)

print("""
🔹 Context Size Impact:

   Context   │ KV Cache per GPU │ Recommended for 70B
   ──────────┼──────────────────┼─────────────────────
   512       │ ~0.5 GB          │ ✅ Safe
   2048      │ ~1.5 GB          │ ✅ Default
   4096      │ ~3 GB            │ ⚠️ Tight fit
   8192      │ ~6 GB            │ ❌ May OOM

🔹 Batch Size Impact:

   n_batch │ Memory │ Speed
   ────────┼────────┼──────
   64      │ Lower  │ Slower
   128     │ Medium │ Medium (recommended)
   256     │ Higher │ Faster
   512     │ High   │ Fastest (may OOM)

🔹 If Running Out of Memory:

   1. Reduce context_size (2048 → 1024)
   2. Reduce n_batch (128 → 64)
   3. Use smaller quantization (IQ3_XS → IQ2_XXS)
   4. Keep some layers on CPU (n_gpu_layers=60)
""")

💡 MEMORY OPTIMIZATION TIPS

🔹 Context Size Impact:

   Context   │ KV Cache per GPU │ Recommended for 70B
   ──────────┼──────────────────┼─────────────────────
   512       │ ~0.5 GB          │ ✅ Safe
   2048      │ ~1.5 GB          │ ✅ Default
   4096      │ ~3 GB            │ ⚠️ Tight fit
   8192      │ ~6 GB            │ ❌ May OOM

🔹 Batch Size Impact:

   n_batch │ Memory │ Speed
   ────────┼────────┼──────
   64      │ Lower  │ Slower
   128     │ Medium │ Medium (recommended)
   256     │ Higher │ Faster
   512     │ High   │ Fastest (may OOM)

🔹 If Running Out of Memory:

   1. Reduce context_size (2048 → 1024)
   2. Reduce n_batch (128 → 64)
   3. Use smaller quantization (IQ3_XS → IQ2_XXS)
   4. Keep some layers on CPU (n_gpu_layers=60)



## Step 13: Cleanup

Stops server and verifies both GPUs have released all VRAM after large model deployment.

Explores split-mode options (layer vs row splitting) for tensor parallelism showing performance implications of different strategies.

In [24]:
print("🛑 Stopping server...")
server.stop_server()
print("✅ Server stopped")

# Check memory freed
!nvidia-smi --query-gpu=index,memory.used,memory.free --format=csv

🛑 Stopping server...
✅ Server stopped
index, memory.used [MiB], memory.free [MiB]
0, 3 MiB, 15093 MiB
1, 3 MiB, 15093 MiB


## 📚 Summary

### Model Recommendations for Kaggle Dual T4:

| Goal | Model | Quantization | VRAM | Speed |
|------|-------|-------------|------|-------|
| Best Speed | 7-8B | Q4_K_M | 5GB | 50+ tok/s |
| Best Balance | 13B | Q4_K_M | 8GB | 30+ tok/s |


### Key Configuration for 70B:
```python
ServerConfig(
    n_gpu_layers=999,
    tensor_split="0.48,0.48",
    context_size=2048,  # Keep small
    flash_attn=True,
    n_batch=128,        # Keep moderate
)
```

### I-Quant Selection:
- **IQ3_XS** - Best quality that fits dual T4
- **IQ2_XXS** - More headroom, lower quality

---

**Next:** [10-complete-workflow](10-complete-workflow-llamatelemetry-v0.1.0.ipynb)

Stops llama-server and verifies both GPUs have released all VRAM after large model deployment testing.